In [276]:
import regex as re

CANONICAL_HEADER_PATTERNS = [
    r'\b(?:work|professional)?\s*experience\b',
    r'\b(?:education(?:\s+(?:and|&)\s+training)?|training(?:\s+(?:and|&)\s+(?:development|certifications?|programs?)|s)?|specialized\s+training)\b',
    r'\b(?:technical\s+)?skills?\b',
    r'\b(?:certifications?|licenses?|certifications?\s*(?:and|&)\s*licenses?)\b',
    r'\b(?:projects?|key\s+projects|project\s+highlights)\b',
    r'\blanguages?\b',
    r'\b(?:executive\s+)?summary\b',
    r'\b(?:professional\s+)?affiliations?\b',
    r'\b(?:awards?|accomplishments?|honors|achievements)\b',
    r'\bpublications?\b',
    r'\b(?:interests|hobbies|extracurricular\s+activities)\b',
    r'\b(?:volunteer\s+experience|community\s+involvement|volunteering)\b',
    r'\bobjective\b',
    r'\breferences\b',
    r'\bprofile\b'
]



def is_known_section_header(line):
    stripped = line.strip()
    return any(
        re.search(rf'^\s*{pattern}\s*[:\-]?\s*$', stripped, re.IGNORECASE)
        for pattern in CANONICAL_HEADER_PATTERNS
    )


def is_date_like(text):
    return bool(re.match(r'(?i)^\d{1,2}/\d{4}$|^[A-Za-z]{3,9} \d{4}$|^\d{4}$', text.strip()))

def is_location_like(text):
    return bool(re.fullmatch(r'(City|State|[A-Z][a-z]{1,15})', text.strip()))

def is_degree_fragment(text):
    return bool(re.fullmatch(r'[A-Za-z]{2,5}', text.strip()))  # BBA, MSc, etc.


def detect_real_headers(lines):
    section_lines = []

    for i, line in enumerate(lines):
        stripped = line.strip()
        if not stripped:
            continue

        # ✅ Canonical match: skip heuristics & filtering
        if is_known_section_header(stripped):
            section_lines.append((i, stripped))
            continue

        # ❌ Filtering for heuristic-only (to avoid false positives)
        if is_date_like(stripped) or is_location_like(stripped) or is_degree_fragment(stripped):
            continue

    return section_lines





In [277]:
def preprocess_resume_text(text: str) -> str:
    import regex as re

    # Clean tabs and escaped slashes
    text = text.replace('\\/', '/').replace('\t', ' ')

    # ✅ Preprocess city/state block BEFORE newline segmentation
    # Collapses things like: "Company Name   －   City  ,   State"
    text = re.sub(
        r'(?i)([A-Za-z0-9&.,()\-\' ]{2,})\s*[-－–—]{1,2}\s*([A-Z][a-z]+)\s*,\s*([A-Z][a-z]+)',
        r'\1 — \2 , \3',
        text
    )

    # ✅ Restore: Convert 3+ spaces into line breaks — segment-based
    text = re.sub(r' {3,}', '\n', text)

    # Normalize excessive newlines and tabs/spaces
    text = re.sub(r'\n{2,}', '\n', text)
    text = re.sub(r'[ \t]+', ' ', text)

    # ✅ Add newlines around known section headers
    def insert_header_breaks(line):
        stripped = line.strip()
        if is_known_section_header(stripped):
            return f"\n{stripped}\n"
        return stripped

    lines = text.splitlines()
    lines = [insert_header_breaks(line) for line in lines]
    text = "\n".join(lines)

    # ✅ Add newlines around date ranges
    text = re.sub(
        r'(?i)(?<!\n)(\d{1,2}/\d{4})\s*(to|-)\s*(\d{1,2}/\d{4}|current|present)',
        r'\n\1 to \3\n',
        text
    )

    return text.strip()


In [278]:
def format_resume_for_llm(text: str) -> str:
    text = preprocess_resume_text(text)
    lines = text.splitlines()
    #lines = merge_company_location(lines)

    headers = detect_real_headers(lines)
    print("Detected Headers:", headers)
    header_indices = {idx for idx, _ in headers}

    output_lines = []
    for i, line in enumerate(lines):
        stripped = line.strip()

        if not stripped:
            continue

        # Skip lines that are isolated numbers (e.g. caused by bad splitting)
        if stripped.isdigit():
            continue

        # Insert a line break before date ranges (if not already done)
        if re.match(r'(?i)^\d{1,2}/\d{4} to (\d{1,2}/\d{4}|current|present)$', stripped):
            output_lines.append("")  # add a break before
            output_lines.append(stripped)
            continue

        # Header detection
        if i in header_indices:
            output_lines.append("")  # add a blank line before header
            output_lines.append(stripped.upper())
            output_lines.append("")  # and after
        else:
            output_lines.append(stripped)

    formatted = "\n".join(output_lines)

    # Normalize inline date ranges (if still embedded)
    formatted = re.sub(
        r'(?i)(\d{1,2}/\d{4})\s*(to|-)\s*(\d{1,2}/\d{4}|current|present)',
        r'\1 to \3',
        formatted
    )

    # Format SKILLS section as bullets
    formatted = re.sub(
        r'(?i)(\nSKILLS\n)([^\n]+)',
        lambda m: m.group(1) + "\n" + "\n".join(f"- {s.strip()}" for s in m.group(2).split(',')),
        formatted
    )

    return formatted.strip()


In [279]:
def format_text(text: str) -> str:
    text = format_resume_for_llm(text)
    print(text)
    #return text        
    

In [280]:
sample_resume = r"""         ACCOUNTANT         Highlights        Microsoft Office :\tIntermediate in all the Microsoft Office components( Excel, Word, PowerPoint, Outlook and Access); Very familiar with ( Macros, V-look ups, SQL, calculating formulas and manipulating reports as well as smart view) Running query reports and creating reports.  SAP PeopleSoft:          Navigate within Resource one to retrieve financial reports and run queries              Experience     08\/2014   to   Current     Accountant    Company Name   －   City  ,   State      An accountant within the Experimental Therapeutics Research department, which prepares grant, restricted and non-restricted financial reports to  PI's within the department.  Create financial models and analysis for all restricted and non-restricted accounts within our department.  Prepares monthly financial reports for PI's.  Reconcile the ncRNA Core on a monthly basis.  Assists with the completion of effort reporting as well as completes Job Data Update Form ( JDUF) via the EPAF system and expense transfers as needed to clear deficit.  Assist with the preparation of operating budget.  Assist with financial model for specific grants within our department.  Monitors and reviews departmental transactions to ensure compliance with established financial controls in accordance with divisional and institutional policies and to ensure applicable revenue and expenses are captured.  Communicates with Grants and Contracts to ensure that requests completed in a timely manner.  Reviews Open PO on a monthly basis and close all PO's as necessary.  Analyzes and evaluates operations of financial systems, prepares recommendations and documents to update fiscal services and other related policies and procedures with respect to operating systems.  Assist with the invoicing via ILABS.  Construct a macro-enabled reconciliation template which significantly reduces keying errors while maximize reconciliation efficiency.  Implemented a macro-enabled reconciliation template, which increased effectiveness by twenty percent and significantly reduced keying errors.  Prepares training materials within the department to increase productivity and standardized processes.  Completes special projects as assigned by the department Administrator.         12\/2012   to   08\/2014     Staff Accountant    Company Name   －   City  ,   State      Full cycle accountant for several operating companies within Sysco Foods Company; completed and issued weekly, monthly, and quarterly financial statements to the operating companies for review and submit to the corporate office.  Uplaod and run reports in business objects for CFO ofoperating company to review.  Verified journal ledger entries of cash and check payments, purchases, expenses and trial balances by examining and authenticating inventory items.  Prepared monthly P&L for several operating companies within the Sysco Corporation and research variances, reviewed transactions in the GL for accuracy, and researched transactions that deviate from the purpose of the account.  Reconciled aging AR transactions and performed financial analysis, identified and explained deviations from planned or historical data.  Assisted in evaluating control systems in the accounting process to ensure operating companies comply with GAAP as well as provided the necessary information to both internal and external auditors on an as needed basis.  Reviewed account reconciliation from other accountants in a timely manner.         01\/2010   to   02\/2012     Accountant \/Auditor    Company Name   －   City  ,   State      Served as both an accountant and auditor within the department of financial services.  The role consisted of utilizing software applications to compile, retrieve, and summarize accounting information for analysis and reporting purposes.  Examined accounting source documents for accuracy, completeness, and compliance with departmental and state rules, regulations, and agreements.  Provided financial trainings for over 240 faculty and staff employees to enforce the rules and regulations of the procurement card set by set the State of Texas procurement regulation.  Maintained controlling records required to ensure accuracy of all data entered into the accounting system.  Identified and reported situations not in compliance with PVAMU internal controls, policies, and procedures, recommending improvements to the accounting process to optimize internal control.  Performed duties in compliance with GAAP, company and department policies and procedures, internal controls and Sarbanes-Oxley requirements.  Reviewed expense report for irregularities and recommended corrective measures to improve internal controls.  Prepared adjusting and closing entries, statements, and analyze financial operations.          Education     May 2011     Masters of Science  :   Accounting    Prairie View A&M University   －   City  ,   State      Accounting       Dec 2009     BBA  :   Accounting    Prairie View A&M University   －   City  ,   State      Accounting        Professional Affiliations    CPA Candidate Participated in various community activities, including the Phi Beta Lambda and  National     Association of Black Accountants Texas Society of CPA's Beta Gamma Sigma Greater Women's Chamber of Society Completed all the Research Operations Curriculum in 2015       Skills    account reconciliation, accounting, accountant, accounting system, AR, budget, business objects, closing, Contracts, financial, financial analysis, financial controls, financial operations, financial reports, financial statements, GL, Grants, inventory, invoicing, ledger, Macros, Access, Excel, Microsoft Office, office, Outlook, PowerPoint, Word, Monitors, operating systems, PeopleSoft, policies, processes, procurement, reporting, Research, SAP, Sarbanes-Oxley, training materials, view   """
format_text(sample_resume)

Detected Headers: [(5, 'Experience'), (29, 'Education'), (44, 'Professional Affiliations'), (49, 'Skills')]
ACCOUNTANT
Highlights
Microsoft Office :\tIntermediate in all the Microsoft Office components( Excel, Word, PowerPoint, Outlook and Access); Very familiar with ( Macros, V-look ups, SQL, calculating formulas and manipulating reports as well as smart view) Running query reports and creating reports. SAP PeopleSoft:
Navigate within Resource one to retrieve financial reports and run queries

EXPERIENCE


8/2014 to Current
Accountant
Company Name
— City , State
An accountant within the Experimental Therapeutics Research department, which prepares grant, restricted and non-restricted financial reports to PI's within the department. Create financial models and analysis for all restricted and non-restricted accounts within our department. Prepares monthly financial reports for PI's. Reconcile the ncRNA Core on a monthly basis. Assists with the completion of effort reporting as well as c

In [281]:
sample_resume = r"""         ADVOCATE       Summary    In-depth knowledge of taking orders, compute charges, and administer billing or payments.\n* Hands-on experience in reviewing, updating or making changes to customer accounts.\n* Track record of listening and responding to customers' needs and concerns.\n* Able to handle returns and complaints effectively.\n* Demonstrated ability to record details of client contacts and service history.\n* Proven ability to determine answers and solutions quickly.\n* Special talent for handling irate and angry customers.\n* Proven written and oral communication skills\n* Proficient in managing business correspondence\n* Ability to organize personal work priorities\n* Knowledge of filing and updating records\n* Ability to work independently and as part of a team\n* Excellent organizational skills\n* Expert in handling office equipment\n* Internet savvy with a proven expertise in using MS Office applications\n* Excellent customer service orientation\n* Special talent for researching and analyzing data effectively\n* Exceptional attention to detail with proven interpersonal skills      Skills          Microsoft Word  Microsoft Office  Microsoft Outlook  Microsoft Power Point  Microsoft Excel  Mostly Microsoft Excel and Microsoft Word                Experience     10\/2016   to   Current     Advocate    Company Name   －   City  ,   State     Respond to and resolve on the first call, customer service inquires and issues by identifying the topic and type of assistance the caller needs such as benefits, eligibility and claims, financial spending accounts and correspondence.  Help guide and educate customers about the fundamentals and benefits of consumer-driven health care topics to include managing their health and well-being by selecting the best benefit plan options, maximizing the value of their health plan benefits and choosing a quality care provider   Intervene with care providers (doctor's offices) on behalf of the customer to assist with appointment scheduling or connections with internal specialists for assistance when needed  Assist customers in navigating myuhc.com and other UnitedHealth Group websites and encourage and reassure them to become self-sufficient  Own problem through to resolution on behalf of the customer in real time or through comprehensive and timely follow-up with the member  Research complex issues across multiple databases and work with support resources to resolve customer issues and\/or partner with others to resolve escalated issues  Provide education and status on previously submitted pre-authorizations or pre-determination requests  Meet the performance goals established for the position in the areas of: efficiency, call quality, customer satisfaction, first call resolution and attendance          06\/2015   to   10\/2016     CSR II    Company Name   －   City  ,   State      Received an award of excellent customer service\nReceived \"Year of service Award\"\nReceived 5 year award\nReceived employee of the month twice with Brookdale Senior Living\nReceived \"Consultant Sales Award\"\nReceived employee recognition 6 months in a row along with a trophy of excellence and 6 other trophies with JP Morgan Chase.  I was a Top Performer for 2014.         10\/2013   to   06\/2015     Business Banker    Company Name   －   City  ,   State      As a Business Banker I provided guidance and assistance to business owners in managing their accounts.  I manage quality assurance, problem resolution, and a lot of data entry.  Worked a lot with numbers.  I am a Peer Coach in which I help train, guide, and motivate new hires.  Responsible for ensuring the client experience is world class and that interactions\/issues are resolved accurately and expediently.  Handle client email\/phone call interactions of a maintenance nature by retaining a thorough knowledge of the Policies and Procedures and all systems in order to assist the client and manage escalated issues.  Use a high level of client service skills in order to defuse potentially difficult and\/or upset customers with complex issues.            Took the initiative to create escalation of any issue that cannot be resolved within the anticipated service level or one that creates risk or exposure for the customer or the bank.  Identify trends within the escalated emails\/issues that lead to improvement opportunities in policies\/procedures processes.  Utilize time at work efficiently to meet or exceed production goals as outlined in the metrics scorecard.  Consistently perform follow up to ensure customer satisfaction and to validate the accuracy of work.  Was required to assist department in meeting and exceeding SLA's by handling other functions when volumes fluctuate.  Research and resolve complex client's issues by working in partnership with partner groups.  Types of cases included fraud resolution, treasury solutions, complex account maintenance, internal relationship maintenance and billing inquiries.  Support field Business Relationship Managers in the successful servicing of customers.  Investigate electronic transactions for successful resolution.  Use multiple systems and internal resources to resolve issues.  Increase customer satisfaction through excellent communication and follow up.  Document issues and record resolutions for the firm's system of record.  Acted independently within policy to assist customers and internal partners and escalate in cases where the policy is preventing us from delivering the right client experience.  Demonstrate sound decision making skills to protect the firm's interest while providing a superior customer experience.         04\/2004   to   09\/2013     Life Enrichment Coordinator\/Marketing Coordinator    Company Name  －   City  ,   State      Designed a creative and exciting life enrichment program to meet the individual needs and interests of the community.  Plan monthly calendar for the community.  Participated in discharge planning with the other members of the management team.  Coordinated the transportation, monitored budget, equipment, and supplies through Microsoft Excel.  Provided guidance and acted as a resource to staff on the importance of life enrichment.  Ensured that programs met all state, federal, local regulations.  Coordinate quarterly events at the community.  Developed and maintained contact with community agencies and supervise volunteers.  Coordinated move-ins and move-outs.  Coordinated the Welcome Committee.  Lots of Data Entry through Microsoft Excel and filing.          Education and Training     2018       Business Administration Human Resources    Argosy University          Business Administration Human Resources       2001       Paralegal Studies    Penn Foster University          Paralegal Studies       1997           Cosmetology College\n          National Beauty College                 1996     High School Diploma  :   Business and Cosmetology    W.W. Samuell High School          Business and Cosmetology        Skills    adding machines, Balance, billing, bonds, budget, Business Administration, calculators, cashier, clerical, Coach, excellent communication, Consultant, client, customer satisfaction, excellent customer service, Data Entry, decision making, discharge planning, email, filing, financial, funds, Human Resource, managing, Microsoft Excel, exchange, mail, money, Microsoft Office, Microsoft Outlook, Microsoft Power Point, Microsoft Word, photography, Policies, problem resolution, processes, purchasing, quality assurance, receiving, maintain records, Research, Sales, SLA, sound, phone, transportation, treasury, type, typing, written   """
format_text(sample_resume)

Detected Headers: [(2, 'Summary'), (6, 'Skills'), (10, 'Experience'), (43, 'Education and Training'), (62, 'Skills')]
ADVOCATE

SUMMARY

In-depth knowledge of taking orders, compute charges, and administer billing or payments.\n* Hands-on experience in reviewing, updating or making changes to customer accounts.\n* Track record of listening and responding to customers' needs and concerns.\n* Able to handle returns and complaints effectively.\n* Demonstrated ability to record details of client contacts and service history.\n* Proven ability to determine answers and solutions quickly.\n* Special talent for handling irate and angry customers.\n* Proven written and oral communication skills\n* Proficient in managing business correspondence\n* Ability to organize personal work priorities\n* Knowledge of filing and updating records\n* Ability to work independently and as part of a team\n* Excellent organizational skills\n* Expert in handling office equipment\n* Internet savvy with a proven ex

In [282]:
sample_resume = r"""         AUTOMOBILE TRANSPORTER       Professional Summary     Dedicated, responsible Class A driver  with a clean driver's license. Self-motivated and customer-focused.Exceptional leader talented at leadership, communication, and management skills. Excellent team player with over 20 years of experience in the military and trucking industry.Talented Mechanist Tech with more than 8 years of experience developing and executing maintenance programs. Excellent troubleshooting skills. Areas of expertise include mechanics, technology, and management.          Key Skills          Quality control  OSHA regulatory compliance  Dependable independent worker  Clean D.O.T.  Good at following instructions      Solid communication skills  Strong organizational skills  Strong work ethic  Team player  Timely project completion            Work Experience     05\/2014   -   Current     Company Name   －   City  ,   State    Automobile Transporter        Conducted daily DOT pre-trip inspections according to a set checklist.  Hooked and unhooked trailers from the tractor.  Coordinated daily delivery schedules based on customer schedules, peak delivery times and alternate routes.  Perform emergency repairs including service hydraulic system.  Maintain accurate records such as vehicle logs, records of cargo and billing statements in accordance with regulations.  Obeyed traffic laws and followed established traffic and transportation procedures  Transported freight from origin to destination in a safe and timely manner.         09\/2009   -   05\/2014     Company Name   －   City  ,   State    Driver\/Car Hauler        Coordinated daily delivery schedules based on customer schedules, peak delivery times and alternate routes.  Conducted daily DOT pre-trip inspections according to a set checklist.  Maintained records required for compliance with state and federal regulations.  Interacted with customers and vendors in a friendly and timely manner.  Transported freight from origin to destination in a safe and timely manner.  Checked load accuracy and stability before each trip.  Picked up customer loads in a timely and accurate manner.  Maintained telephone and radio contact with supervisor to receive delivery instructions.  Conducted emergency roadside repairs, including changing tires, replacing light bulbs and installing fuses and tire chains.  Recorded expenses and maintained receipts.         02\/2006   -   09\/2009     Company Name   －   City  ,   State    Independent Contractor         Facilitated successful internal and external audits through sound and thorough documentation.  Composed effective accounting reports summarizing accounts payable data.  Posted receipts to appropriate general ledger accounts.  Reconciled discrepancies between accounts receivable general ledger account and accounts receivable trial balance account.  Balanced monthly general ledger accounts to accurately record cost and month end accruals.  Supervised invoice processing, purchase orders, expense reports, credit memos and payment transactions.  Process accounts payable for 3 employees.  Entered financial data into the company accounting database to be verified and reconciled.  Tracked financial progress by creating quarterly and yearly balance sheets.  Streamlined bookkeeping procedures to increase efficiency and productivity.  Developed monthly, quarterly and annual profit and loss statements and balance sheets.  Performed periodic budgeting\/modeling to project monthly cash requirements.  Drove car transporter, applying knowledge of commercial driving regulations in order to assure prompt delivery to dealerships.  Verified the contents of inventory to match bill of lading forms.  Maintain driver log.         10\/2004   -   02\/2006     Company Name   －   City  ,   State    Driver\/Car Hauler        Transport automobiles from manufacturing plant to the dealership.  Worker directly with training, departments, co-signees, and management to achieve on prompt deliveries new and pre-owned vehicles.  Perform emergency repairs including service hydraulic system.  Maintain accurate records such as vehicle logs, records of cargo and billing statements in accordance with regulations.  Obeyed traffic laws and followed established traffic and transportation procedures.  Trained New drivers.         08\/2003   -   10\/2004     Company Name   －   City  ,   State    Driver\/Car Hauler        Coordinated daily delivery schedules based on customer schedules, peak delivery times and alternate routes.  Obtained and maintained proper delivery authorization and pickup documentation.  Conducted daily DOT pre-trip inspections according to a set checklist.  Loaded and unloaded trailers with mechanical freight handling equipment.  Delivered personal Vehicles to homes and places of business within established time frames.  Interacted with customers and vendors in a friendly and timely manner.         09\/2000   -   05\/2003     Company Name   －   City  ,   State    Driver        Safely loaded and unloaded deliveries according to size of load and content description.  Transported deliveries locally and over the road.  Managed discrepancy documentation for incoming shipments.  Submitted all delivery documentation in a timely manner.  Maintained a daily, legible DOT log book and submitted corresponding documents.  Completed basic maintenance such as minor repairs to keep vehicle neat and running properly.  Reported all accidents, damage and malfunctions involving company equipment to management.  Inspected the truck for defects and safe operating condition before, during and after trips.  Established and maintained excellent customer relationships.  Answered customer questions regarding delivery promptly and accurately.  Loaded and unloaded freight to assure safety and minimize risk of damage and dangerous conditions.         04\/1992   -   09\/2000     Company Name   －   City  ,   State    Aviation Machinist Mate        Supervised the creation and validation of anti-terrorism plans on all ships.  Developed and led training programs in preparation for combat.  Operated and maintained communications equipment.  Documented and processed classified materials.  Recommended retransmission equipment employment based on capabilities of the equipment and the mission requirements.  Guided and coordinated unit's force protection programs to meet fleet requirements.  Planned and briefed personnel on missions.  Established and commanded field communication operations.  Routinely checked measuring equipment to resolve testing problems.  Monitored production operations for compliance with specifications and promptly reported defects.  Discarded and rejected products, materials and equipment that did not meet specifications.  Worked within flight operations to maintain a culture of safety with efficient procedures.  Prepared and filed flight plans both domestic and international, utilizing all available resources and aircraft capabilities for maximum efficiency.  Ensured that all aircraft operational matters were within manufacturers, FAA and company limits including fuel loading, weight and balance and performance measures.  Coordinated communication between air traffic control and maintenance personnel.  Performed and supervised airfield management activities.  Monitored gauges, warning devices and control panels to verify aircraft performance and to regulate engine speed.  Calculated the aircraft weight and balance, performance and fuel requirements.  Assisted with the Safety Risk Management (SRM) process.  Announced and demonstrated safety and emergency procedures.          Specialized Training     2000     Road Master Truck driving  School   －   City  ,   State  ,   United States      Truck Driving    GPA:   GPA: 3.7     Class A CDL License             Road Master Truck Driving School  City  ,   State      Class \"A\" Commercial Driver License (CDL)        Credentials     TWIC card    """
format_text(sample_resume)

Detected Headers: [(7, 'Work Experience'), (59, 'Specialized Training')]
AUTOMOBILE TRANSPORTER
Professional Summary
Dedicated, responsible Class A driver with a clean driver's license. Self-motivated and customer-focused.Exceptional leader talented at leadership, communication, and management skills. Excellent team player with over 20 years of experience in the military and trucking industry.Talented Mechanist Tech with more than 8 years of experience developing and executing maintenance programs. Excellent troubleshooting skills. Areas of expertise include mechanics, technology, and management.
Key Skills
Quality control OSHA regulatory compliance Dependable independent worker Clean D.O.T. Good at following instructions
Solid communication skills Strong organizational skills Strong work ethic Team player Timely project completion

WORK EXPERIENCE


5/2014 to Current
Company Name
— City , State
Automobile Transporter
Conducted daily DOT pre-trip inspections according to a set checklis

In [ ]:
sample_resume = r""" """
format_text(sample_resume)

In [ ]:
sample_resume = r""" """
format_text(sample_resume)